In [3]:
import pandas as pd
import re
df = pd.read_csv("/Users/morenx/Downloads/mt/arxiv.csv")

In [2]:
# new columns
df = df[["submitter",
        "title", 
        "abstract", "categories"]]
df

,submitter,title,abstract,categories
0,Pavel Nadolsky,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,hep-ph
1,Louis Theran,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-...",math.CO cs.CG
2,Hongjun Pan,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...,physics.gen-ph
3,David Callan,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...,math.CO
4,Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...,math.CA math.FA
...,...,...,...,...
1747302,Ruslan Prozorov,On the origin of the irreversibility line in t...,We report on measurements of the angular dep...,supr-con cond-mat.supr-con
1747303,Durga P. Choudhury,Nonlinear Response of HTSC Thin Film Microwave...,The non-linear microwave surface impedance o...,supr-con cond-mat.supr-con
1747304,Durga P. Choudhury,Critical State Flux Penetration and Linear Mic...,The vortex contribution to the dc field (H) ...,supr-con cond-mat.supr-con
1747305,Hasegawa Yasumasa,Density of States and NMR Relaxation Rate in A...,We show that the density of states in an ani...,supr-con cond-mat.supr-con


In [3]:
"""
esta tercero version del mapeo de categorias, fusionare la clase economics con quantitative finance, tambien hare
una traduccion de subcategorias antiguas a la moderna, en mi primer testeo de mi sistema me dio un error de 0 falsos
"""
def map_categoryv3(categories_str):
    if not isinstance(categories_str, str):
        return "Unknown"
    primary_label = categories_str.split(' ')[0]
    
    # "TRADUCCIÓN" (Antiguo -> Moderno)
    legacy_mapping = {
        # Matemáticas
        'alg-geom': 'math.AG',
        'funct-an': 'math.FA',
        'q-alg':    'math.QA',
        'dg-ga':    'math.DG',
        'geo-alg':  'math.AG',
        
        # Computer Science
        'cmp-lg':   'cs.CL', # Antes "Computation and Language" no tenía prefijo cs
        
        # Física / No Lineal
        'adap-org': 'nlin.AO',
        'comp-gas': 'nlin.CG',
        'chao-dyn': 'nlin.CD',
        'patt-sol': 'nlin.PS',
        'solv-int': 'nlin.SI',
        'supr-con': 'cond-mat.supr-con',
        'acc-phys': 'physics.acc-ph',
        'chem-ph':  'physics.chem-ph',
        'mat-ph':   'math-ph', 
    }
    # Si la etiqueta es de las viejas, la actualizamos a la moderna 
    if primary_label in legacy_mapping:
        primary_label = legacy_mapping[primary_label]
    
    # prefijos estandar
    if primary_label.startswith('cs.'): return 'Computer Science'
    if primary_label.startswith('math.'): return 'Mathematics'
    if primary_label == 'math-ph': return 'Mathematics'
    if primary_label.startswith('stat.'): return 'Statistics'
    if primary_label.startswith('q-bio.'): return 'Quantitative Biology'
    if primary_label.startswith('eess.'): return 'Electrical Engineering'

    # Fusión Econ & Fin
    if primary_label.startswith('q-fin.') or primary_label.startswith('econ.'):
        return 'Economics & Finance'

    # Prefijos Física
    # Nota: agregamos nlin y physics explícitamente
    physics_prefixes = [
        'astro-ph', 'cond-mat', 'gr-qc', 'hep-ex', 'hep-lat', 'hep-ph',
        'hep-th', 'nucl-ex', 'nucl-th', 'physics', 'quant-ph', 'nlin'
    ]
    
    if any(primary_label.startswith(p) for p in physics_prefixes):
        return 'Physics'

    # Reglas residuales (Ya casi no quedan, solo bayes-an)
    if primary_label == 'bayes-an': return 'Statistics'
    
    return 'Other'

# Aplicar la función al DataFrame
df['macro_cat'] = df['categories'].apply(map_categoryv3)

In [4]:
print(df['macro_cat'].value_counts())

macro_cat
Physics                   1039837
Mathematics                390284
Computer Science           244029
Statistics                  29340
Quantitative Biology        19886
Electrical Engineering      14153
Economics & Finance          9504
Other                         274
Name: count, dtype: int64


In [5]:
df['macro_cat'] = df['categories'].apply(map_categoryv3)

df['primary_cat'] = df['categories'].apply(lambda x: x.split(' ')[0] if isinstance(x, str) else 'Unknown')

#quitare los papers que no pude categorizar en macro categoria
df_clean = df[df['macro_cat'] != 'Other'].copy()

In [6]:
print(df_clean['macro_cat'].value_counts())

macro_cat
Physics                   1039837
Mathematics                390284
Computer Science           244029
Statistics                  29340
Quantitative Biology        19886
Electrical Engineering      14153
Economics & Finance          9504
Name: count, dtype: int64


PRIMARY_CAT (NIVEL 2(ESPECIFICAS)) PARA EL MODELO B
MACRO_CAT NIVEL (1 (GENERAL)) PARA MODELO A


### SEGUNDO RE SAMPLING( CAPPING )

LAS CLASES MAYORITARIAS LAS BAJARE A LAS CLASES MEDIANAS, PARA QUE LA ESCALA DE PROPORCIONES NO ESTE TAN MAL VARIADA.

In [7]:
##limite maximo de clase
CAP_LIMIT = 20000
print(" recorte (Capping) a las clases mayoritarias...")
def cap_class(group):
    # Si la clase tiene más de 20k, tomamos una muestra aleatoria de 20k
    if len(group) > CAP_LIMIT:
        return group.sample(n=CAP_LIMIT, random_state=42)
    # Si tiene menos (ej. Economics), devolvemos todos los registros tal cual
    return group

# Aplicamos la función a cada grupo (macro_cat)
df_balanced = df_clean.groupby('macro_cat').apply(cap_class).reset_index(drop=True)

 recorte (Capping) a las clases mayoritarias...


/var/folders/lt/n0_y0qx173v13mss419tfhlm0000gn/T/ipykernel_8956/2772801574.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df_clean.groupby('macro_cat').apply(cap_class).reset_index(drop=True)


In [8]:
df_balanced

,submitter,title,abstract,categories,macro_cat,primary_cat
0,Fredrik Bagge Carlson,On the Calibration of Force/Torque Sensors in ...,We present and analyze methods for the kinem...,cs.RO,Computer Science,cs.RO
1,Sourav Medya,K-Core Minimization: A Game Theoretic Approach,K-cores are maximal induced subgraphs where ...,cs.SI cs.DS,Computer Science,cs.SI
2,EDA Publishing Association,Assertion-Based Design Exploration of DVS in N...,With the scaling of technology and higher re...,cs.AR,Computer Science,cs.AR
3,Vuong M. Ngo,WordNet-Based Information Retrieval Using Comm...,Text search based on lexical matching of key...,cs.CL cs.IR,Computer Science,cs.CL
4,Xianwen Wang,Detecting and Tracking The Real-time Hot Topic...,"In this study, following the idea of our pre...",cs.DL cs.IR,Computer Science,cs.DL
...,...,...,...,...,...,...
123538,"Samuel M\""uller",Robust model selection in generalized linear m...,"In this paper, we extend to generalized line...",stat.ME,Statistics,stat.ME
123539,Assad Oberai,Bayesian Inference with Generative Adversarial...,Bayesian inference is used extensively to in...,stat.ML cs.LG eess.IV physics.comp-ph,Statistics,stat.ML
123540,Solt Kov\'acs,Seeded intervals and noise level estimation in...,"In this discussion, we compare the choice of...",stat.ME stat.CO,Statistics,stat.ME
123541,Alexander Ratner,Learning to Compose Domain-Specific Transforma...,Data augmentation is a ubiquitous technique ...,stat.ML cs.CV cs.LG,Statistics,stat.ML


In [9]:
print("NUEVA DISTRIBUCIÓN DEL DATASET")
print(df_balanced['macro_cat'].value_counts())

print(f"\nTamaño total del nuevo dataset: {df_clean.shape[0]}")

NUEVA DISTRIBUCIÓN DEL DATASET
macro_cat
Computer Science          20000
Mathematics               20000
Physics                   20000
Statistics                20000
Quantitative Biology      19886
Electrical Engineering    14153
Economics & Finance        9504
Name: count, dtype: int64

Tamaño total del nuevo dataset: 1747033


In [10]:
!pip install spacy

In [11]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [12]:
import spacy 
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
class ArxivCleaner:
    def __init__(self):
        # cargar spacy
        try:
            self.nlp = spacy.load("en_core_web_sm", disable=['ner', 'lemmatizer'])
        except OSError:
            print("Modelo no encontrado.")
            raise

        #stopwords a mantener
        scientific_keep = {
            "however", "therefore", "thus", "hence", "consequently", 
            "furthermore", "moreover", "nevertheless", "meanwhile", 
            "otherwise", "via", "whereas", "whereby", "propose", "method",
            "using", "present", "significant", "analysis"
        }
        self.custom_stopwords = STOP_WORDS - scientific_keep
        
        # --- REGEX ---
        self.url_pattern = re.compile(r'http\S+|www\S+|https\S+')
        self.doi_pattern = re.compile(r'\b(10[.][0-9]{4,}(?:[.][0-9]+)*/(?:(?!["&\'<>])\S)+)\b')
        self.ref_pattern = re.compile(r'\[\d+(?:-\d+)?(?:,\s*\d+)*\]')
        
        # LaTeX: Bloques entre $...$ o $$...$$ (soporta saltos de linea con DOTALL)
        self.latex_pattern = re.compile(r'\$\$?.*?\$\$?', re.DOTALL)
        
        # Comandos LaTeX sueltos (ej: \sigma, \alpha, \frac)
        self.latex_command_pattern = re.compile(r'\\[a-zA-Z]+')

    def _clean_text_single(self, text):
        if not text or not isinstance(text, str):
            return ""

        #  BORRADO DE LATEX (Reemplazo por espacio en blanco)
        text = self.latex_pattern.sub(' ', text)          
        text = self.latex_command_pattern.sub(' ', text) 
        
        # limpieza dois y urls
        text = self.url_pattern.sub('', text)
        text = self.doi_pattern.sub('', text)
        text = self.ref_pattern.sub('', text)
        
        # quitamos caracteres no ascii
        text = re.sub(r'[^\x00-\x7F]+', ' ', text)
        
        # espacios dobles
        text = re.sub(r'\s+', ' ', text).strip()

        # procesamiento spacy
        doc = self.nlp(text)
        cleaned_tokens = []

        for token in doc:
            if token.is_space: continue
            if token.text.lower() in self.custom_stopwords: continue
            # Conservar guiones intra-palabra
            if token.is_punct and not (token.text == '-' and token.i > 0 and token.i < len(doc)-1): continue
                
            word = token.text
            
            # Eliminar símbolos matemáticos sueltos que hayan quedado (+, =, ^, >)
            if word in {'+', '=', '^', '>', '<', '~', '|', '*', '%'}:
                continue

            # Lógica de mayúsculas/números
            if word.isupper() and len(word) > 1:
                final_word = word
            elif token.like_num:
                final_word = word
            else:
                final_word = word.lower()
            
            # Filtro de ruido: palabras de 1 letra (excepto 'a' o 'i' o números)
            if len(final_word) < 2 and final_word not in ['a', 'i'] and not final_word.isdigit():
                continue
            
            if final_word.strip():
                cleaned_tokens.append(final_word)

        return " ".join(cleaned_tokens)

    def process_dataframe(self, df, output_col='cleaned_text'):
        if 'title' not in df.columns or 'abstract' not in df.columns:
            raise ValueError("Faltan columnas 'title' o 'abstract'")
        
        print("Limpiando texto formal (papers arxiv)...")
        df_clean = df.copy()
        
        # Concatenación Title + . + Abstract
        df_clean['temp_raw'] = (
            df_clean['title'].fillna('').astype(str).str.strip() + 
            ". " + 
            df_clean['abstract'].fillna('').astype(str).str.strip()
        )
        
        df_clean[output_col] = df_clean['temp_raw'].apply(self._clean_text_single)
        df_clean.drop(columns=['temp_raw'], inplace=True)
        
        print(f"Listo. Columna '{output_col}' creada.")
        return df_clean


In [14]:
cleaner = ArxivCleaner()
cleaned = cleaner.process_dataframe(df_balanced)
cleaned.to_csv("cleanedv3.csv", index=False)
print("Proceso terminado.")

Limpiando texto formal (papers arxiv)...
Listo. Columna 'cleaned_text' creada.
Proceso terminado.


In [4]:
CLEANED_PATH = "/Users/morenx/Downloads/mt/codigos/cleanedv3.csv"
cleaned = pd.read_csv(CLEANED_PATH)

In [5]:
cleaned

,submitter,title,abstract,categories,macro_cat,primary_cat,cleaned_text
0,Fredrik Bagge Carlson,On the Calibration of Force/Torque Sensors in ...,We present and analyze methods for the kinem...,cs.RO,Computer Science,cs.RO,calibration force torque sensors robotics pres...
1,Sourav Medya,K-Core Minimization: A Game Theoretic Approach,K-cores are maximal induced subgraphs where ...,cs.SI cs.DS,Computer Science,cs.SI,core minimization game theoretic approach core...
2,EDA Publishing Association,Assertion-Based Design Exploration of DVS in N...,With the scaling of technology and higher re...,cs.AR,Computer Science,cs.AR,assertion based design exploration DVS network...
3,Vuong M. Ngo,WordNet-Based Information Retrieval Using Comm...,Text search based on lexical matching of key...,cs.CL cs.IR,Computer Science,cs.CL,wordnet based information retrieval using comm...
4,Xianwen Wang,Detecting and Tracking The Real-time Hot Topic...,"In this study, following the idea of our pre...",cs.DL cs.IR,Computer Science,cs.DL,detecting tracking real time hot topics study ...
...,...,...,...,...,...,...,...
123538,"Samuel M\""uller",Robust model selection in generalized linear m...,"In this paper, we extend to generalized line...",stat.ME,Statistics,stat.ME,robust model selection generalized linear mode...
123539,Assad Oberai,Bayesian Inference with Generative Adversarial...,Bayesian inference is used extensively to in...,stat.ML cs.LG eess.IV physics.comp-ph,Statistics,stat.ML,bayesian inference generative adversarial netw...
123540,Solt Kov\'acs,Seeded intervals and noise level estimation in...,"In this discussion, we compare the choice of...",stat.ME stat.CO,Statistics,stat.ME,seeded intervals noise level estimation change...
123541,Alexander Ratner,Learning to Compose Domain-Specific Transforma...,Data augmentation is a ubiquitous technique ...,stat.ML cs.CV cs.LG,Statistics,stat.ML,learning compose domain specific transformatio...
